In [1]:
import polars as pl

In [2]:
df = pl.DataFrame({'keys': list('aab'), 'vals': [10, 7, 1]})
df

keys,vals
str,i64
"""a""",10
"""a""",7
"""b""",1


In [5]:
df.groupby('keys', maintain_order=True).agg([
    pl.col('vals').map(lambda s: s.shift()).alias('shiftmap'),
    pl.col('vals').shift().alias('shiftexpr')])

keys,shiftmap,shiftexpr
str,list[i64],list[i64]
"""a""","[null, 10]","[null, 10]"
"""b""",[7],[null]
